# Naver Diabetes Community Webcrawler

## importing libraries

In [2]:
import requests
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import time
from random import randint
import re
import pandas as pd
import pickle
from tqdm import tqdm
import os

from random import *
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import numpy as np

import winsound as sd


wd = webdriver.Chrome(r"C:\Users\MinSeong\Desktop\chrome driver\chromedriver.exe")
wait = WebDriverWait(wd, 5)

## Fetching post urls from Boards

### 1.Approach board url

In [86]:
#url을 가져와서 뷰숲 돌리는게 빠름. Complexity!!



# 상담실 url
all_couns = 'https://cafe.naver.com/dangsamo?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10096425%26search.menuid=73%26search.boardtype=L%26search.totalCount=151%26search.page=' 
noob_couns = 'https://cafe.naver.com/dangsamo?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10096425%26search.menuid=682%26search.boardtype=L%26search.totalCount=151%26search.page='
pre_couns = 'https://cafe.naver.com/dangsamo?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10096425%26search.menuid=688%26search.boardtype=L%26search.totalCount=151%26search.page=' #내당능장애


#친목 url
type_chat = 'https://cafe.naver.com/dangsamo?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10096425%26search.menuid=3%26search.boardtype=L%26search.totalCount=151%26search.page='# 유형 별 당건수다
meal = 'https://cafe.naver.com/dangsamo?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10096425%26search.menuid=285%26search.boardtype=C%26search.totalCount=101%26search.page='
exerc = 'https://cafe.naver.com/dangsamo?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10096425%26search.menuid=330%26search.boardtype=C%26search.totalCount=101%26search.page='

board_urls = [all_couns,  noob_couns, pre_couns, type_chat, meal, exerc]
# 수집할 페이지 목록
pages = [1000,130,79,999,1000,851] # 각 게시판. #분석에 방해되는 형식 쓴 예전 글은 제외함.

## 2. Approach post url

In [51]:
#원래 코드. 됨.

# 피클 저장할 경로
import os
os.chdir(r'C:\Users\MinSeong\Desktop\수업\비즈니스 텍마\Term Proj')

urls = []

i = 5 #0~5 
print(f"{i}번째 게시판 진행 중")
time.sleep(0.2)


for page in tqdm(range(pages[i]+1)): #url접근
    # 게시판 페이지 접근
    wd.get(board_urls[i] + str(page)) # 페이지 돌아가며 접근하기.
    
    time.sleep(random.random())

    # frame전환
    wd.switch_to.frame('cafe_main')
    bs = BeautifulSoup(wd.page_source, "html.parser")

    # 게시글의 url가져오기
    # url href를 가져와서
   
    #cont = bs.find_all('div',{'class':'inner_list'}) #보통은 이거로
    #cont = bs.find_all('div',{'class':'con'}) #meal은 이거로
    cont = bs.find_all('div',{'class':'con_top'})#exec은 이거로
    
    
    for c in cont:
        url = 'https://cafe.naver.com' + str(c.select('div > a')[0].attrs['href']) # meal은 태그 잘못 됨.
        urls.append(url)

# Beep when the code is done
fr = 130
dr = 1001
sd.Beep(fr,dr)

#https://m.blog.naver.com/PostView.nhn?blogId=bosongmoon&logNo=221595169882&proxyReferer=https:%2F%2Fwww.google.com%2F
#참조함

5번째 게시판 진행 중


NameError: name 'pages' is not defined

In [94]:
# 저장 신중히
print(i,"번째 게시판")

5 번째 게시판


신중히 저장

In [95]:
#urls 피클 저장 신중히
name = 'exerc'

with open(name,'wb') as fw:
    pickle.dump(urls,fw)

In [96]:
#urls피클 로드
with open('exerc','rb') as fr:
    u = pickle.load(fr)

###  Iframe에서 원본 url 가져오기 --> 복잡. Selenium으로 우회.

# 3. Fetching data from posts

In [2]:
# Empty df


col_name = ['title','content','writer','date','view','likes','no_comments','board','url']
df = pd.DataFrame([],columns = [col_name])

import os
os.chdir(r'C:\Users\MinSeong\Desktop\수업\비즈니스 텍마\Term Proj')

In [3]:
# 가져올 게시판:
i = 5 #0~5 번째 게시판 #0,1,2,3,4,5번완료 
board_names = ['all_couns',  'noob_couns', 'pre_couns', 'type_chat', 'meal', 'exerc']
url_skip = [81,62,61,61,0,0]
#urls피클 로드
with open(board_names[i],'rb') as fr:
    urls = pickle.load(fr)

# 공지 데이터 제거

urls = urls[url_skip[i]:] # 80까지는 상담실 유형 게시판 공지 url. 보이는거랑 다르게. 실제 데이터 보니 공지임.


#urls = urls[24:] # 24까지는 친목 유형 게시판 공지 url

print(i,"번째 게시판",board_names[i],'진행중',len(urls),'개 url.')

5 번째 게시판 exerc 진행중 8520 개 url.


df_name 바꾸기

In [5]:
#완성본. 


#iframe에 있는 진짜 url 가져오기

#wd = webdriver.Chrome(r"C:\Users\MinSeong\Desktop\chrome driver\chromedriver.exe")


troublesome_url = []

for index,url in tqdm(enumerate(urls)):
    try:
        wd.get(url)
        wait.until(EC.presence_of_all_elements_located)
        time.sleep(random.random()+0.8)
        wd.switch_to.frame('cafe_main')
        soup = BeautifulSoup(wd.page_source,'html.parser')

        #print(url)

        # title
        title = np.nan
        try:
            title = soup.select('h3.title_text')[0].text
        except:
            title = np.nan
        #print(title)
        # content # 다시 봐야함
        content = np.nan
        
        try:
            content = soup.select('div.se-module.se-module-text')[0].text
        except:
            try:
                content = soup.select('div.ContentRenderer > div > p')[0].text
                #print('작동',content)
                #print(url)
                
                    
            except:
                try:
                    contents= soup.select('div.NHN_Writeform_Main > div >div > b >span')
                    for c in contents:
                        empty.append(c.text)
                        content = "".join(empty)
                except:
                    try:
                        contents= soup.select('div.NHN_Writeform_Main > div')
                        empty = []
                        for c in contents:
                            empty.append(c.text)
                            content = "".join(empty)
                    except:
                        print("content nan!")
                        print(url)
                        content = np.nan

        #print(content)
        #writer    
        writer = np.nan
        try:
            writer = soup.select('div.nick_box > a')[0].text
        except:
            writer = np.nan
        #print(writer)
        #date
        date = np.nan
        try:
            date = soup.select('span.date')[0].text
            date = date.strip()
        except:
            date = np.nan
        #print(date)

        #view
        view = np.nan
        try:
            view_rec = soup.select('span.count')[0].text
            #print(view_rec)
            p_view = re.compile('조회 ([0-9]*,?[0-9]*)')
            view = p_view.search(view_rec).group(1)
        except:
            view = np.nan
        #print(view)

        #likes
        likes = np.nan
        try:
            likes = soup.select('em.u_cnt._count')[0].text

        except:
            likes = np.nan
        #print(likes)

        #no_comments
        no_comments = np.nan
        try:
            no_comments = soup.select('strong.num')[0].text # 두개 select됨. 같은값.
        except:
            no_comments = np.nan
        #print(no_comments)

        #board
        board = board_names[i]
        #print(board)

    except:
        troublesome_url.append(url) # 삭제된 게시물
        
        continue
    df.loc[index] = title,content,writer,date,view,likes,no_comments,board,url

8520it [3:31:20,  1.49s/it]


In [8]:
file_name = board_names[i]+'.csv'
df.to_csv(file_name)

In [6]:
board_names[i]

'exerc'

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8508 entries, 0 to 8519
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   (title,)        8478 non-null   object
 1   (content,)      6638 non-null   object
 2   (writer,)       8478 non-null   object
 3   (date,)         8478 non-null   object
 4   (view,)         8478 non-null   object
 5   (likes,)        8478 non-null   object
 6   (no_comments,)  8478 non-null   object
 7   (board,)        8508 non-null   object
 8   (url,)          8508 non-null   object
dtypes: object(9)
memory usage: 664.7+ KB


In [9]:
len(df['content'].value_counts())

5225

# 4) df 합쳐주기

In [3]:
df1 = pd.read_csv('df_1.csv') 
df2 = pd.read_csv('df_2_4100.csv') #all_couns 0

noob_couns = pd.read_csv('noob_couns.csv') #1

pre_couns = pd.read_csv('pre_couns.csv') #2

type_chat = pd.read_csv('type_chat.csv') #3

meal = pd.read_csv('meal.csv') # 4

exerc = pd.read_csv('exerc.csv')#5




In [6]:

df = pd.concat([df1,df2,noob_couns,pre_couns,type_chat,meal,exerc],axis = 0)
df = df.drop(['Unnamed: 0',	'Unnamed: 0.1'], axis = 1)

In [8]:
df.to_csv('diabetes.csv') #크롤링 완료. 결과물.

In [3]:
import os
import pandas as pd
os.chdir(r'C:\Users\MinSeong\Desktop\수업\비즈니스 텍마\Term Proj')


df = pd.read_csv('diabetes.csv')

In [7]:
df['board'].value_counts() #5만 레코드

all_couns     14984
type_chat     14966
meal          10000
exerc          8508
noob_couns     1934
pre_couns      1181
Name: board, dtype: int64